In [70]:
import pandas as pd
import json
import ast  # For safely parsing strings to Python objects


In [71]:
df = pd.read_csv('TRADES_CopyTr_90D_ROI.csv') #load the data
print(df.info())
print(df.describe())
print(df.head())
df = df.dropna()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
           Port_IDs
count  1.500000e+02
mean   3.975655e+18
std    6.887470e+16
min    3.672755e+18
25%    3.943672e+18
50%    3.998616e+18
75%    4.029508e+18
max    4.041860e+18
              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


In [72]:

def parse_trade_history(trade_history_str): 
    try:
        # Convert the string to a list of dictionaries
        trades = ast.literal_eval(trade_history_str)
        
        # Extract relevant data fields
        extracted_data = {
            'side': [trade['side'] for trade in trades],
            'price': [trade['price'] for trade in trades],
            'quantity': [trade['quantity'] for trade in trades],
            'realizedProfit': [trade['realizedProfit'] for trade in trades]
        }
        return extracted_data
    except Exception as e:
        print(f"Error parsing trade history: {e}")
        return None
df['Trade_History'] = [parse_trade_history(trade) for trade in df['Trade_History']]
df


,Port_IDs,Trade_History
0,3925368433214965504,"{'side': ['BUY', 'BUY', 'BUY', 'BUY', 'BUY', '..."
1,4002413037164645377,"{'side': ['SELL', 'BUY', 'BUY', 'BUY', 'BUY', ..."
2,3923766029921022977,"{'side': ['SELL', 'SELL', 'SELL', 'SELL', 'SEL..."
3,3994879592543698688,"{'side': ['SELL', 'SELL', 'SELL', 'SELL', 'SEL..."
4,3926423286576838657,"{'side': ['SELL', 'SELL', 'SELL', 'BUY', 'BUY'..."
...,...,...
145,4000222729738650369,"{'side': ['SELL', 'SELL', 'SELL', 'SELL', 'SEL..."
146,3998659472131949824,"{'side': ['SELL', 'SELL', 'SELL', 'BUY', 'BUY'..."
147,4028701921959171840,"{'side': ['BUY', 'BUY', 'BUY', 'BUY', 'BUY', '..."
148,4014818740371615232,"{'side': ['BUY', 'SELL', 'SELL', 'SELL', 'SELL..."


In [73]:

df.reset_index(drop=True, inplace=True)
metrics={
"Port_IDs":[] , 
"Trade_History":[],
"pnl":[],
"roi":[],
"sharpe_ratio":[],
"mdd":[],
"win_positions":[],
"total_positions":[],
"win_rate":[],
}

def calculate_metrics(trade,Port_ID):
    trade = pd.DataFrame(trade)
    metrics["Port_IDs"].append(Port_ID)
    metrics["Trade_History"].append(trade)
    
# 1. Total Investment (sum of prices for all BUY trades)
    trade['investment'] = trade.apply(lambda x: x['price'] * x['quantity'] if x['side'] == 'BUY' else 0, axis=1)
    total_investment = trade['investment'].sum()

# 2. Total Net Profit (sum of realizedPnl for all SELL trades)
    total_net_profit = trade.loc[trade['side'] == 'SELL', 'realizedProfit'].sum()
    metrics["pnl"].append(total_net_profit)
    

# 3. Calculate Total ROI
    if total_investment > 0:
        total_roi = (total_net_profit / total_investment) * 100
    else:
        total_roi = 0  # Avoid division by zero if there's no investment
    metrics["roi"].append(total_roi)
# 4. PnL (Profit and Loss for each trade)
    trade['Pnl']= trade.apply(lambda x: (x['price'] * x['quantity']) if x['side'] == 'SELL' else 0, axis=1) - trade['investment']

# 5. Sharpe Ratio
    returns = trade['Pnl'] / total_investment  # Returns per trade
    risk_free_rate = 0.01  # Example risk-free rate (1%)
    metrics["sharpe_ratio"].append( (returns.mean() - risk_free_rate) / returns.std())

# 6. Maximum Drawdown (MDD)
    trade['cumulative_return'] = (1 + returns).cumprod()
    trade['cumulative_max'] = trade['cumulative_return'].cummax()
    trade['drawdown'] = (trade['cumulative_return'] - trade['cumulative_max']) / trade['cumulative_max']
    metrics["mdd"].append( trade['drawdown'].min())

# 7. Win Positions (trades with positive PnL)
    metrics["win_positions"].append( trade[trade['Pnl'] > 0].shape[0])

# 8. Total Positions (total number of trades)
    metrics["total_positions"].append( trade.shape[0])

# 9. Win Rate
    metrics["win_rate"].append((metrics["win_positions"][-1] / metrics["total_positions"][-1]) * 100)

# Calculate ROI for each Port_ID and merge it back to the original DataFrame
for i in range(len(df)):
    calculate_metrics(df['Trade_History'][i], df["Port_IDs"][i])
    
df=pd.DataFrame(metrics)
df

,Port_IDs,Trade_History,pnl,roi,sharpe_ratio,mdd,win_positions,total_positions,win_rate
0,3925368433214965504,side price quantity realizedP...,2627.648039,0.000021,-1.699814,-0.169417,591,1458,40.534979
1,4002413037164645377,side price quantity realizedProfi...,1592.844019,0.002085,-0.324095,-0.594029,286,576,49.652778
2,3923766029921022977,side price quantity realizedProfi...,353.994040,0.000006,-0.767120,-0.255864,742,1259,58.935663
3,3994879592543698688,side price quantity realizedPro...,2618.176363,0.000457,-0.585791,-0.319048,428,653,65.543645
4,3926423286576838657,side price quantity realizedProf...,1308.689960,0.000013,-0.518469,-0.357787,164,478,34.309623
...,...,...,...,...,...,...,...,...,...
144,4000222729738650369,side price quantity realizedProf...,253.767180,0.000022,-0.642527,-0.308131,268,958,27.974948
145,3998659472131949824,side price quantity realizedProfit...,765.567930,0.000006,-0.475010,-0.176538,85,147,57.823129
146,4028701921959171840,side price quantity realizedProfit ...,5074.836199,0.000004,-1.644992,-0.245703,306,687,44.541485
147,4014818740371615232,side price quantity realizedPro...,1098.976340,0.000008,-0.721489,-0.183356,231,529,43.667297


In [74]:
weights = {
    'roi': 0.25,
    'pnl': 0.25,
    'sharpe_ratio': 0.20,
    'mdd': -0.15, 
    'win_rate': 0.10,
    'win_positions': 0.05,
    'total_positions': 0.10
}
df['Score'] = (df['roi'] * weights['roi'] + df['pnl'] * weights['pnl'] +
               df['sharpe_ratio'] * weights['sharpe_ratio'] +
               df['win_rate'] * weights['win_rate'] + df['mdd'] * weights['mdd'] + 
               df['win_positions'] * weights['win_positions'] +
               df['total_positions'] * weights['total_positions']
              )


df['Rank'] =df['Score'].rank(ascending=False)
df.to_csv('final_output.csv', index=False)
df = df.sort_values(by='Rank')
df

,Port_IDs,Trade_History,pnl,roi,sharpe_ratio,mdd,win_positions,total_positions,win_rate,Score,Rank
64,3960874214179953664,side price quantity realizedProfit...,16456.159208,0.000010,-2.987410,-0.076795,1295,2391,54.161439,4422.719986,1.0
22,3956076827719377409,side price quantity realized...,11734.319436,0.000271,-1.731192,-0.641798,2268,3916,57.916241,3444.121582,2.0
43,3788465932399412480,side price quantity realizedPr...,11026.652637,0.002685,-0.981568,-0.467754,1323,3069,43.108504,3133.898531,3.0
60,3936410995029308417,side price quantity realizedProfit...,10681.514000,0.000099,0.227723,-0.658105,9,20,45.000000,2677.472785,4.0
5,3987739404272887297,side price quantity realizedProfit...,6427.010841,0.026833,-2.523967,-0.334377,2936,6050,48.528926,2362.957674,5.0
...,...,...,...,...,...,...,...,...,...,...,...
19,4041804592937345281,side price quantity realizedProf...,-838.901660,-0.000106,-0.439404,-0.550307,144,368,39.130435,-161.817732,145.0
47,3995532094997544704,side price quantity realizedP...,-6219.209513,-0.000006,-4.115538,-0.097108,3035,6051,50.156999,-793.745221,146.0
142,4008804082027254529,side price quantity realizedProf...,-3788.851090,-0.000006,-1.177546,-0.162485,423,975,43.384615,-824.435449,147.0
76,4000877324693233921,side price quantity realizedProfit...,-8051.142985,-0.014936,-1.442648,-0.469536,2975,6050,49.173554,-1254.340224,148.0


In [75]:
top_20_accounts = df.groupby('Port_IDs')['Score'].mean().nlargest(20)

top_20_accounts.to_csv('top_20_output.csv')

top_20_accounts


Port_IDs
3960874214179953664    4422.719986
3956076827719377409    3444.121582
3788465932399412480    3133.898531
3936410995029308417    2677.472785
3987739404272887297    2362.957674
3939318616482048768    2085.278308
3818233536529843712    1820.582523
3942630767220672256    1800.353178
3951723256343550465    1791.282379
4011626972687000576    1625.805609
3931992636670880512    1534.110860
3976088733682659585    1474.836212
4031493134338259457    1471.513540
4037121179310444801    1467.002876
3886752488982104320    1465.444736
3957509601040585217    1430.644931
4028701921959171840    1356.871056
4008711265867865600    1335.480601
3953764659057081857    1308.613442
3879821005658659073    1180.131782
Name: Score, dtype: float64